# 6. Spam classification

## (a)

In [1]:
import collections

import numpy as np

import src.util as util
import src.svm as svm

Process a message into a list of lowercase words:

In [2]:
def get_words(message):
    """Get the normalized list of words from a message string.

    This function should split a message into words, normalize them, and return
    the resulting list. For splitting, you should split on spaces. For normalization,
    you should convert everything to lowercase.

    Args:
        message: A string containing an SMS message

    Returns:
       The list of normalized words from the message.
    """

    # *** START CODE HERE ***
    return message.lower().split()
    # *** END CODE HERE ***

Test it:

In [3]:
get_words("teST bla bLa 32 23")

['test', 'bla', 'bla', '32', '23']

Get a dictionary translating common words to integers:

In [4]:
def create_dictionary(messages):
    """Create a dictionary mapping words to integer indices.

    This function should create a dictionary of word to indices using the provided
    training messages. Use get_words to process each message. 

    Rare words are often not useful for modeling. Please only add words to the dictionary
    if they occur in at least five messages.

    Args:
        messages: A list of strings containing SMS messages

    Returns:
        A python dict mapping words to integers.
    """

    # *** START CODE HERE ***
    words = [word for message in messages for word in get_words(message)]
    
    word_counter = collections.Counter(words)
    common_words = [word for word in word_counter if word_counter[word] >= 5]
    return {word:i for i,word in enumerate(common_words)}
    # *** END CODE HERE ***

Test it:

In [5]:
messages = ["hi tom", "bye peter", "bye"] + 5*["hello"] + 6*["hi"] + 6*["moin"]
dic = create_dictionary(messages)
dic

{'hi': 0, 'hello': 1, 'moin': 2}

Encode messages into a numpy array:

In [6]:
def transform_text(messages, word_dictionary):
    """Transform a list of text messages into a numpy array for further processing.

    This function should create a numpy array that contains the number of times each word
    appears in each message. Each row in the resulting array should correspond to each 
    message and each column should correspond to a word.

    Use the provided word dictionary to map words to column indices. Ignore words that 
    are not present in the dictionary. Use get_words to get the words for a message.

    Args:
        messages: A list of strings where each string is an SMS message.
        word_dictionary: A python dict mapping words to integers.

    Returns:
        A numpy array marking the words present in each message.
    """
    # *** START CODE HERE ***
    word_counters = [collections.Counter(get_words(message)) for message in messages]
    
    m = len(messages)
    n = len(word_dictionary)
    res = np.zeros((m,n), dtype=int)
    for i in range(m):
        for word,j in word_dictionary.items():
            if word in word_counters[i]:
                res[i,j] = word_counters[i][word]
    
    return res
    # *** END CODE HERE ***

Load training, valuation and test sets:

In [7]:
train_messages, train_labels = util.load_spam_dataset('data/ds6_train.tsv')
val_messages, val_labels = util.load_spam_dataset('data/ds6_val.tsv')
test_messages, test_labels = util.load_spam_dataset('data/ds6_test.tsv')

Create a dictionary from the training set and use it to encode all messages:

In [8]:
dictionary = create_dictionary(train_messages)

train_matrix = transform_text(train_messages, dictionary)
val_matrix = transform_text(val_messages, dictionary)
test_matrix = transform_text(test_messages, dictionary)

train_matrix[:100,:]

array([[1, 1, 1, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0],
       ...,
       [0, 1, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0]])

## (b)

Messages in the dataset are represented by a vector $x^{(i)} \in \mathbb N^n$, where $n$ is the number of words in our dictionary and where $x^{(i)}_k$ is the number of occurences of word $k$ in the $i$-th message.

Therefore the maximum likelihood parameters for our model with Laplace smoothing are given by:

$$\begin{align*} 
\phi_{k\mid y=1} &= \frac{1+\sum_{i=1}^m x^{(i)}_k  1\{y^{(i)}=1\}}{n+\sum_{i=1}^m 1\{y^{(i)}=1\}\sum_{j=1}^n x^{(i)}_j} \\
\phi_{k\mid y=0} &= \frac{1+\sum_{i=1}^m x^{(i)}_k  1\{y^{(i)}=0\}}{n+\sum_{i=1}^m 1\{y^{(i)}=0\}\sum_{j=1}^n x^{(i)}_j} \\
\phi_y &= \frac 1m \sum_{i=1}^m1\{y^{(i)}=1\}
\end{align*}$$

where $y^{(i)}$ is the label of message $x^{(i)}$.

In [9]:
def fit_naive_bayes_model(matrix, labels):
    """Fit a naive bayes model.

    This function should fit a Naive Bayes model given a training matrix and labels.

    The function should return the state of that model.

    Feel free to use whatever datatype you wish for the state of the model.

    Args:
        matrix: A numpy array containing word counts for the training data
        labels: The binary (0 or 1) labels for that training data

    Returns: The trained model
    """

    # *** START CODE HERE ***
    x = matrix
    y = labels
    m,n = x.shape


    
    phi_1 = (1+ np.sum(x[y==1], axis = 0)) / (n + np.sum(x[y==1]))
    phi_0 = (1+ np.sum(x[y==0], axis = 0)) / (n + np.sum(x[y==0]))
    phi_y = np.mean(labels)
    
    return (phi_1, phi_0, phi_y)
    # *** END CODE HERE ***

Because we assume that for a message $x\in \mathbb N^n$ given its label $y\in \{0,1\}$ is multinomially distributed, we get:

$$ p(x\mid y ) = \frac {\left(\sum_{k=1}^n x_k\right)!}{\prod_{k=1}^n x_k!} \prod_{k=1}^n \phi_{k\mid y}^{x_k}.$$

Because 
$$ p(y\mid x) = \frac{p(x\mid y)p(y)}{p(x)},$$

we have 
$$ \frac{p(y=1\mid x)}{p(y=0\mid x)} = \frac{p(x\mid y=1)\phi_y}{p(x\mid y=0) (1-\phi_y) }$$

We will classify $x$ as spam if the above fraction is $\geq 1$, which is equivalent to 

$$\begin{align*}
\log \frac{1-\phi_y}{\phi_y} &\leq \log \frac{p(x\mid y=1)}{p(x\mid y=0)}\\
&= \log \prod_{k=1}^n\phi^{x_k}_{k\mid y=1}\phi^{-x_k}_{k\mid y=0}\\
&= \sum_{k=1}^n x_k \left(\log \phi_{k\mid y=1} - \log \phi_{k\mid y=0}\right).
\end{align*}$$

In [10]:
def predict_from_naive_bayes_model(model, matrix):
    """Use a Naive Bayes model to compute predictions for a target matrix.

    This function should be able to predict on the models that fit_naive_bayes_model
    outputs.

    Args:
        model: A trained model from fit_naive_bayes_model
        matrix: A numpy array containing word counts

    Returns: A numpy array containg the predictions from the model
    """
    # *** START CODE HERE ***
    phi_1, phi_0, phi_y = model
    x = matrix
    m,n = x.shape
    
    boundary = np.log(1-phi_y) - np.log(phi_y)
    
    return boundary <= (x @ (np.log(phi_1) - np.log(phi_0)))
    # *** END CODE HERE ***

Fit a model to the training set and compute its predictions on the test set:

In [11]:
naive_bayes_model = fit_naive_bayes_model(train_matrix, train_labels)
naive_bayes_predictions = predict_from_naive_bayes_model(naive_bayes_model, test_matrix)

Compute the model's accuracy:

In [12]:
naive_bayes_accuracy = np.mean(naive_bayes_predictions == test_labels)
print('Naive Bayes had an accuracy of {} on the testing set'.format(naive_bayes_accuracy))

Naive Bayes had an accuracy of 0.978494623655914 on the testing set


## (c)

In our notation we have
$$ \begin{align*}
\log \frac{p(x_j=i\mid y=1)}{p(x_j=i\mid y=0} &= \log \frac{\phi_{i\mid y=1}}{\phi_{i\mid y=0}}
\end{align*}$$

In [13]:
def get_top_five_naive_bayes_words(model, dictionary):
    """Compute the top five words that are most indicative of the spam (i.e positive) class.

    Ues the metric given in 6c as a measure of how indicative a word is.
    Return the words in sorted form, with the most indicative word first.

    Args:
        model: The Naive Bayes model returned from fit_naive_bayes_model
        dictionary: A mapping of word to integer ids

    Returns: The top five most indicative words in sorted order with the most indicative first
    """
    # *** START CODE HERE ***
    phi_1, phi_0, _ = model
    token_ratings = np.log(phi_1/phi_0)
    top_five_tokens = np.argpartition(token_ratings,-5)[-5:]
    return [ word for word in dictionary if dictionary[word] in top_five_tokens]
    # *** END CODE HERE ***

Now we can find the most indicative words:

In [14]:
top_5_words = get_top_five_naive_bayes_words(naive_bayes_model, dictionary)

print('The top 5 indicative words for Naive Bayes are: ', top_5_words)

The top 5 indicative words for Naive Bayes are:  ['claim', 'tone', 'urgent!', 'won', 'prize']


## (d)

In [15]:
def compute_best_svm_radius(train_matrix, train_labels, val_matrix, val_labels, radius_to_consider):
    """Compute the optimal SVM radius using the provided training and evaluation datasets.

    You should only consider radius values within the radius_to_consider list.
    You should use accuracy as a metric for comparing the different radius values.

    Args:
        train_matrix: The word counts for the training data
        train_labels: The spma or not spam labels for the training data
        val_matrix: The word counts for the validation data
        val_labels: The spam or not spam labels for the validation data
        radius_to_consider: The radius values to consider
    
    Returns:
        The best radius which maximizes SVM accuracy.
    """
    # *** START CODE HERE ***
    accuracies = []
    
    for radius in radius_to_consider:
        pred = svm.train_and_predict_svm(train_matrix, train_labels, test_matrix=val_matrix, radius=radius)
        accuracies.append(np.mean(pred == val_labels))
    
    return radius_to_consider[np.argmax(accuracies)]
    # *** END CODE HERE ***

Find the best radius and compute the test accuracy of an RBF-SVM model with that radius:

In [16]:
    optimal_radius = compute_best_svm_radius(train_matrix, train_labels, val_matrix, val_labels, [0.01, 0.1, 1, 10])
    print('The optimal SVM radius was {}'.format(optimal_radius))

    svm_predictions = svm.train_and_predict_svm(train_matrix, train_labels, test_matrix, optimal_radius)

    svm_accuracy = np.mean(svm_predictions == test_labels)

    print('The SVM model had an accuracy of {} on the testing set'.format(svm_accuracy, optimal_radius))

The optimal SVM radius was 0.1
The SVM model had an accuracy of 0.9695340501792115 on the testing set
